# Ustawianie środowiska pod RNN

## Konfiguracja integracji z Dyskiem Google



In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive -o nonempty

## Instalacja PyTorch

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

# !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision

import torch

## Instalacja bibliotek

In [0]:
!pip install unidecode tqdm dill pysmi visdom

## Instalacja biblioteki PyDrive wrapper & import 

In [0]:
# This only needs to be done once per notebook.
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

from pathlib import Path
import os

def download_and_save(file_name, file_id):
  downloaded = drive.CreateFile({'id': file_id})
  # fetch file
  downloaded.FetchContent()
  # write file
  with open(file_name,'wb') as f:
       f.write(downloaded.content.read())
      
  print(f'Saved {file_name}')
  
FILE_NAME, FILE_ID = ['file_name', 'file_id']

# Pobranie danych

In [0]:
from pathlib import Path
dataset_path = Path('/content/'); dataset_path  #/ 'data/rnn_generator'; dataset_path
# !mkdir -p $dataset_path
!ls -lh $dataset_path/
# tmp_path = dataset_path / 'tmp/'
# !mkdir -p $tmp_path

## Pobranie datasetu data-names

In [0]:
# data-names.tgz, https://drive.google.com/file/d/17tzhbLwlfdIsimJDTNghFfx22cT9n7OB
names = {FILE_NAME: 'data-names.tgz', FILE_ID: '17tzhbLwlfdIsimJDTNghFfx22cT9n7OB'}
names_path = dataset_path / names[FILE_NAME]

download_and_save(names_path, names[FILE_ID])

In [0]:
# unpack

!tar -xf $names_path -C $dataset_path

In [0]:
!ls -lh $dataset_path/data/names

# Monitorowanie maszyny wirtualnej

In [0]:
!uptime

# start visdom server

In [0]:
!python -m visdom.server -p 8890